In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
# initailzing a list to collect all reviews
name = []
# initailzing a list to collect all reviews
reviews = []

# initailzing a list of travellers type all reviews
travellers_type = []

# initailzing a list of seat type all reviews
seat_type = []

# initailzing a list of route all reviews
route = []

# initailzing a list of rcountry all reviews
locations = []

# initailzing a list to collect star ratings
stars = []

# initialzing a list to collect date of review
date_of_review = []

# initialzing a list to collect date of travel
date_of_travel = []

# initialzing a list to collect country information where is user is from
country = []

In [3]:
url = 'https://www.airlinequality.com/airline-reviews/british-airways/page/1'

In [4]:
def get_next_page(url):
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        pagination = soup.find("article", class_="comp comp_reviews-pagination querylist-pagination position-")
        if pagination:
            anchor_tags = pagination.find_all('a')
            for anchors in anchor_tags:
                if anchors.get('class'):
                    current_page = anchors.text
                    next_page = str(int(current_page)+1)
                    url = url.replace(current_page , next_page)
                    return url
        else:
             return None


In [5]:
url = 'https://www.airlinequality.com/airline-reviews/british-airways/page/1'
while url:
    print('reading url: ', url)
    new_url = get_next_page(url)
    if new_url:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        body = soup.find_all('div',class_='body')
        for lines in body:
            # print(lines)
            try:
                tt = lines.find('td', class_="review-rating-header type_of_traveller").parent
                tt = tt.text
                travellers_type.append(tt[len('Type Of Traveller'):])
            except:
                travellers_type.append('NaN')
            try:
                st = lines.find('td', class_="review-rating-header cabin_flown").parent
                st = st.text
                seat_type.append(st[len("Seat Type"):])

            except:
                seat_type.append('NaN')

            try:
                rf = lines.find('td', class_="review-rating-header route").parent
                rf = rf.text
                route.append(rf[len("Route"):])
            except:
                route.append('NaN')
            try:
                dt = lines.find('td', class_="review-rating-header date_flown").parent
                dt = dt.text
                date_of_travel.append(dt[len("Date Flown"):])
            except:
                date_of_travel.append('NaN')
            dr = lines.find("time", itemprop="datePublished").text if lines.find("time", itemprop="datePublished") else 'NaN' 
            date_of_review.append(dr)
            name_span = lines.find('span', itemprop='name') if lines.find('span', itemprop='name') else 'NaN'
            name.append(name_span.text)
            h3_tag = lines.find("h3", class_="text_sub_header userStatusWrapper")
            location = h3_tag.contents[2].strip(" ()")
            locations.append(location)
            review = lines.find('div', class_='text_content', itemprop='reviewBody')
            reviews.append(review.text)

    for item in soup.find_all("div", class_ = "rating-10"):
        try:
            stars.append(item.span.text)
        except:
            print(f"Error on page {new_url}")
            stars.append("None")

    for item in soup.find_all("h3"):
        country.append(item.span.next_sibling.text.strip(" ()"))
    url = new_url

reading url:  https://www.airlinequality.com/airline-reviews/british-airways/page/1
reading url:  https://www.airlinequality.com/airline-reviews/british-airways/page/2
reading url:  https://www.airlinequality.com/airline-reviews/british-airways/page/3
reading url:  https://www.airlinequality.com/airline-reviews/british-airways/page/4
reading url:  https://www.airlinequality.com/airline-reviews/british-airways/page/5
reading url:  https://www.airlinequality.com/airline-reviews/british-airways/page/6
reading url:  https://www.airlinequality.com/airline-reviews/british-airways/page/7
reading url:  https://www.airlinequality.com/airline-reviews/british-airways/page/8
reading url:  https://www.airlinequality.com/airline-reviews/british-airways/page/9
reading url:  https://www.airlinequality.com/airline-reviews/british-airways/page/10
reading url:  https://www.airlinequality.com/airline-reviews/british-airways/page/11
reading url:  https://www.airlinequality.com/airline-reviews/british-airwa

In [6]:
#check the length of total reviews extracted
print('reviews',len(reviews))
print('name',len(name))
print('travellers_type',len(travellers_type))
print('seat_type', len(seat_type))
print('route', len(route))
print('locations', len(locations))
print('stars',len(stars))
print('date_of_review',len(date_of_review))
print('date_of_travel', len(date_of_travel))
print('country',len(country))

reviews 3550
name 3550
travellers_type 3550
seat_type 3550
route 3550
locations 3550
stars 3916
date_of_review 3550
date_of_travel 3550
country 3560


In [7]:

len(country)

3560

In [8]:
#check the length 
print(len(stars))
stars = stars[:len(reviews)]
country = country[:len(reviews)]
print(len(stars),len(country))

3916
3550 3550


In [9]:
#create  a dataframe from these collected lists of data
data = {"reviews":reviews,
        'name':name,
        "stars": stars, 
        "date_of_review":date_of_review, 
        "country": country,
        'locations':locations,
        'date_of_travel':date_of_travel,
        'travel_type':travellers_type,
        'seat_type':seat_type,
        'route ':route}
df = pd.DataFrame(data)
df.head()

,reviews,name,stars,date_of_review,country,locations,date_of_travel,travel_type,seat_type,route
0,✅ Trip Verified |. The BA first lounge at Term...,E Michaels,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,22nd May 2023,United Kingdom,United Kingdom,May 2023,Business,Business Class,London Heathrow to Malaga
1,Not Verified | Paid a quick visit to Nice yest...,Steve Bennett,2,22nd May 2023,United Kingdom,United Kingdom,May 2023,Couple Leisure,Business Class,London to Nice
2,✅ Trip Verified | Words fail to describe this...,N Mayle,4,19th May 2023,United States,United States,September 2022,Solo Leisure,Business Class,London to San Francisco
3,✅ Trip Verified | Absolutely terrible experie...,E Heale,2,17th May 2023,United States,United States,April 2023,Solo Leisure,Economy Class,London to Dallas
4,✅ Trip Verified | BA overbook every flight to ...,H Mike,1,17th May 2023,United Kingdom,United Kingdom,May 2023,Business,Economy Class,London to Madrid


In [10]:
df.shape

(3550, 10)

# Export the data into a csv format

In [11]:
import os

cwd = os.getcwd()
df.to_csv(cwd+ "/BA_reviews.csv")